In [1]:
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType, StructType, StringType, IntegerType, FloatType, StructField
from pyspark.sql.functions import lit

In [2]:
ss = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("us_import_sample") \
    .getOrCreate()

In [3]:
contact_schema = StructType([
    StructField("identifier", StringType()),
    StructField("name", StringType()),
    StructField("address_1", StringType()),
    StructField("address_2", StringType()),
    StructField("address_3", StringType()),
    StructField("address_4", StringType()),
    StructField("city", StringType()),
    StructField("state_province", StringType()),
    StructField("zip_code", StringType()),
    StructField("country_code", StringType()),
    StructField("contact_name", StringType()),
    StructField("comm_number_qualifier", StringType()),
    StructField("comm_number", StringType())
])

In [4]:
consignee = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .csv(
        './ams/2020/202001201500/ams__consignee_2020__202001201500.csv', 
        schema=contact_schema
    )
consignee = consignee.withColumn('contact_type', lit('consignee'))

In [5]:
consignee.printSchema()

root
 |-- identifier: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address_1: string (nullable = true)
 |-- address_2: string (nullable = true)
 |-- address_3: string (nullable = true)
 |-- address_4: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- comm_number_qualifier: string (nullable = true)
 |-- comm_number: string (nullable = true)
 |-- contact_type: string (nullable = false)



In [6]:
notified_party = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .csv(
        './ams/2020/202001201500/ams__notifyparty_2020__202001201500.csv', 
        schema=contact_schema
    )
notified_party = notified_party.withColumn('contact_type', lit('notified_party'))

In [7]:
shipper = ss.read \
    .option("header", True) \
    .option("escape", '"') \
    .csv(
        './ams/2020/202001201500/ams__shipper_2020__202001201500.csv', 
        schema=contact_schema
    )
shipper = shipper.withColumn('contact_type', lit('shipper'))

In [8]:
contact = consignee.union(shipper).union(notified_party)

In [9]:
contact.printSchema()

root
 |-- identifier: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address_1: string (nullable = true)
 |-- address_2: string (nullable = true)
 |-- address_3: string (nullable = true)
 |-- address_4: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- comm_number_qualifier: string (nullable = true)
 |-- comm_number: string (nullable = true)
 |-- contact_type: string (nullable = false)



In [10]:
contact.coalesce(1).write.mode('overwrite').format("csv") \
    .option("header", True) \
    .option("escape", '"') \
    .save("./ams/contact")